<center>
    <h1> INF285 - Computación Científica </h1>
    <h1> Tarea N°2 V2.0</h1>
    
</center>



# Instrucciones
* La tarea es individual. Sin embargo se invita a todos l@s estudiantes a que discutan entre ustedes las preguntas, pero luego implementen de forma individual su tarea.
* Las consultas sobre las tareas se deben realizar por medio de la plataforma Aula.
* La tarea debe ser realizada en `Jupyter Notebook` (`Python3`).
* Se evaluará la correcta utilización de librerías `NumPy`, `SciPy`, entre otras, así como la correcta implementación de algoritmos de forma vectorizada.
* **No modifique la firma de las funciones** (a menos que se le diga lo contrario) y respete el output que se le exije. **En caso de no respetar esta regla la función se considerará errónea y obtendrá la nota 0.**
* **Asegúrese de que su notebook se ejecute de forma correcta en el orden de las celdas establecido.** Una forma de verificar esto es reiniciar el kernel del notebook y ejecutar todas las celdas nuevamente. **En caso de que su notebook no logre ejecutarse de esta forma se evaluará con nota 0.**  
*  **El archivo de entrega debe denominarse ROL-tarea-numero.ipynb**. _De no respetarse este formato existirá un descuento de **50 puntos**_.
* **No realice prints ni pida inputs**, entregue solo las funciones solicitadas. _De no respetarse este formato existirá un descuento de 50 puntos._
* No se revisarán funciones incompletas.
* No se revisarán tareas fuera de plazo.
* Tareas que demoren más de 2 minutos en ejecutarse recibirán nota 0.
* La fecha de entrega es el Jueves 3 de Junio a las **18:00 hrs**.  
* Debe citar cualquier código ajeno utilizado (incluso si proviene de los Jupyter Notebooks del curso).
* Puede agregar funciones extras siempre y cuando **no interfieran en las firmas** de las funciones principales establecidas.



# Introducción

En esta tarea estudiaremos la descomposición de valores singulares, o también conocida como la factorización matricial SVD.
Lo interesante de esta factorización matricial es que siempre existe para matrices de cualquier dimensión, sean cuadradas o no.
La factorización SVD tiene la siguiente forma para una matriz $A\in\mathbb{R}^{m\times n}$:
$$A = U\,\Sigma\,V^{*},$$
donde $U$ es una matriz unitaria, $\Sigma=\mbox{diag}(\sigma_1,\sigma_2,\dots)$ es un matriz diagonal con $\sigma_i$ el $i$-ésimo valor sigular que tradicionalmente cumplen que $\sigma_i>\sigma_{i+1}$, $V$ es una matriz unitaria, y $^*$ es el operador de transpuesta-conjugada.
En el anexo A de los apuntes del curso se presenta una explicación más detallada de la SVD, la cual debe estudiarse en detalle para poder realizar la tarea.

La tarea se centrará en la aplicación de reducción de dimensionalidad con la SVD.
El algoritmo que se estudiará es el "Análisis de Componentes Principales", o PCA del inglés.
En el anexo B de los apuntes del curso se presenta la relación entre la SVD y PCA.

El contexto con el cual se trabajará será con una pista de audio, a la cual se le aplicará una reducción de dimensionalidad y se comparará la cantidad de memoria requerida en almacenar el archivo original vs la versión comprimida, además de la ''calidad'' de la pista de audio obtenida.

Lamentablemente uno no obtiene buenos resultados si aplica reducción de dimensionalidad separando la pista de audio en segmentos de audio mas cortos y luego aplicar PCA, sin embargo se puede utilizar la ''Short-time Fourier Transform'' (STFT) para pre-procesar la pista de audio y luego ''comprimir'' la pista de audio.
¡Los resultados son de una excelente calidad! Si es que se aplica la reducción de dimensionalidad de forma adecuada.

## Breve descripción de la STFT

La STFT (Short-time Fourier Transform) corresponde a una transformación utilizada para determinar la frecuencia sinusoidal y la fase de secciones locales de una señal que cambia a lo largo del tiempo. Para tiempos discretos (que es nuestro caso) se calcula de la siguiente manera:

$$STFT\{x[n]\}(m,w) = \sum_{n = -\infty}^{\infty}{x[n]w[n-m]e^{-j\omega n}}$$

Donde:

* $w[n-m]$ corresponde a una función ventana (por defecto scipy usa Hann Window)
* $x[n]$ corresponde a la señal a ser transformada
* $\omega$ corresponde a la frecuencia
* j corresponde a la unidad imaginaria

Esta transformación trabaja el audio segmentándolo y aplicando Transformada de Fourier Rápida para cada uno de los segmentos, entregando como resultado una matriz compleja. Esta explicación de STFT es bien breve ya que se utilizará scipy para calcularla (no tendrán que implementarla). Puede revisar el siguiente enlace para mayor información sobre la <a href = "https://en.wikipedia.org/wiki/Short-time_Fourier_transform"> STFT </a> 

En esta tarea se buscará comprimir un archivo de audio *WAV* a costo de reducir levemente su calidad, el cual consiste en una secuencia de datos muestreados en una tasa llamada _sample rate_.
Dado que *SVD* y *PCA* son utilizados tradicionalmente para reducir la dimensionalidad de datos en forma de vectores, primero tendremos que representar el audio como un conjunto de vectores de datos. 
Para esto se utilizará la *STFT* (Short-time Fourier Transform), y obtener de esta la matriz de datos reducir. 
En esta tarea se busca aprender aplicar *PCA con SVD*.

## Flujo de trabajo de la compresión/descompresión de la señal

El flujo que seguirá la tarea será el siguiente:

### Compresión
* 1.- Aplicar STFT al audio obteniendo $M$.
* 2.- Aplicar Split a M obteniendo $M\_Re$ y $M\_Im$.
* 3.- Aplicar PCA a $M\_Re$ y $M\_Im$ con $m1$ y $m2$ componentes, respectivamente.

### Descompresión
* 1.- Reconstruir las matrices $M\_Re$ y $M\_Im$ luego de aplicar PCA.
* 2.- Aplicar Merge a las nuevas matrices $M\_Re$ y $M\_Im$ para obtener la matriz M modificada.
* 3.- Aplicar ISTFT a la matriz M modificada para recuperar el formato de audio (arreglo de datos).

In [ ]:
import numpy as np
import IPython
import time
from scipy.io import wavfile
from scipy.signal import stft
from scipy.signal import istft

In [ ]:
# Esta función de no debe ser modificada.
def WAV_to_Array(WAV_file):
    """
    Parameters
    ----------
    WAV_file             : string 
                           WAV path

    Returns
    -------
    sample_rate          : int
                           Number of samples per second of the WAV file
    data                 : 1-D array                                       
                          NumPy array with WAV file data                   
    """                                                                    
    sample_rate, data = wavfile.read(WAV_file)
    if(len(data.shape) > 1): # En caso de que tenga más de un canal de audio, se trabaja con el primero
        data = data[:,0]
    
    return sample_rate, data

In [ ]:
# Esta función de no debe ser modificada.
def play_audio(data, sample_rate):
    """
    Parameters
    ----------
    data            : 1-D array                                       
                    NumPy array with WAV file data
    sample_rate     : int
                    Number of samples per second of the WAV file
    
    """
    IPython.display.display(IPython.display.Audio(data, rate=sample_rate))
    return

In [ ]:
sample_rate, data = WAV_to_Array("original.wav")
play_audio(data, sample_rate)

A partir de la lectura del archivo WAV, debemos utilizar el contenido de este audio y representarlo como una matriz de datos para aplicar el análisis de componentes principales. 

Antes de reducir la dimensionalidad, necesitamos pre-procesar la señal, para lo cual vamos a utilizar STFT para obtener la matriz de datos $M \in \mathbb{C}^{p\times q}$, donde $p$ y $q$ dependerán de los parámetros utilizados en la STFT, como el intervalo de tiempo en que fue aplicada la STFT a cada segmento y el tamaño de la ventana utilizada en estos (Para esta tarea se utilizará el método de STFT e ISTFT de **scipy.signal** con los valores de parámetros por defecto).

Una vez obtenida la matriz de datos M, esta se separará en su parte real y parte imaginaria, obteniendo finalmente las matrices *M_Re* y *M_Im*, a las cuales se le aplicará PCA con la SVD.


----
## Funciones a Implementar

**(15 puntos)** 1. Crear la función ```Split(M)``` que recibe la matriz compleja ```M``` generada por STFT y retorne las matrices ```M_Re``` y ```M_Im``` resultantes de separar la parte real de la imaginaria de la matriz M. También, deberá crear la función ```Merge(M_Re, M_Im)``` que recibe las matrices reales ```M_Re``` y ```M_Im``` y retorne la matriz compleja ```M``` como resultado de juntar ambas matrices como parte real y parte imaginaria respectivamente. 
Además, a continuación se entregan las funciones ```STFT``` e ```ISTFT``` a utilizar para aplicar la transformación y su inversa respectivamente, las cuales no deben modificarse.

In [23]:
# Esta función de no debe ser modificada.
def STFT(data):
    """
    Parameters
    ----------
    data                 : 1-D array                                      
                          NumPy array with WAV file data
    Returns
    -------
    M                : (p, q) array
                       M complex matrix
    """
    _, _, M = stft(data)
    return M
    
# Esta función de no debe ser modificada.
def ISTFT(M):
    """
    Parameters
    ----------
    M                : (p, q) array
                       M complex matrix
    Returns
    -------
    data                 : 1-D array                                      
                          NumPy array with WAV file data
    """
    _, data = istft(M)
    return data

In [24]:
def Split(M):
    """
    Parameters
    ----------
    M                : (p, q) array
                       M complex matrix
    Returns
    -------
    M_Re             : (p, q) array
                       M_Re real matrix
    M_Im             : (p, q) array
                       M_Im real matrix
    """
    M_Re = np.real(M)
    M_Im = np.imag(M)
    return M_Re, M_Im

def Merge(M_Re, M_Im):
    """
    Parameters
    ----------
    M_Re             : (p, q) array
                       M_Re real matrix
    M_Im             : (p, q) array
                       M_Im real matrix
    Returns
    -------
    M                : (p, q) array
                       M complex matrix
    """
    M = M_Re + M_Im*1j
    return M

### PCA
**(30 puntos)** 2. Implementar la función ```PCA_SVD(M, m)``` que reciba una matriz $M$ real y las $m$ componentes que se utilizarán para comprimir utilizando PCA. La función debe retornar las matrices $V$ y $Y$, y el vector de medias $\boldsymbol \mu$. Además, implementar la función ```PCA_M(V, Y, mu)``` que recibe las matrices generadas por *PCA* y retorne la reconstrucción $M_m$ de la matriz original $M$. 
Para esto debe utilizar la función de SVD de numpy.linalg. Ponga atención si va a requerir la versión _reducida_ o _full_ de la SVD.

In [25]:
# M to PCA
def PCA_SVD(M, m):
    
    #
    
    """
    Parameters
    ----------
    M             : (p, q) array
                    M real matrix
    m              : int
                    Number of components
    Returns
    -------
    V             : (q, m)-array
                     first m principal components
    Y              : (p,m)-array
                     Principal Component Coefficients
    mu             : (q)-array
                     Average per column 
    """
    mu = np.mean(M, axis = 0)
    Z = M - mu
    # Vemos si es necesario usar SVD completo o no
    rows,columns = M.shape
    
    # si m > min(rows,columns), debe usarse SVD completo
    if m > min(rows,columns):
        u, sigma, vh = np.linalg.svd(Z, full_matrices=True)
    else:
        u, sigma, vh = np.linalg.svd(Z, full_matrices=False)
    
    #np.linalg.svd nos entrega V transpuesta, por lo que debemos volver a transponerla
    V = vh.T   
    
    #Y = ZV
    Y = np.dot(Z,V)
    
    Y = Y[:, :m]
    V = V[:, :m]
    return  V, Y, mu

# PCA to 'modified' M
def PCA_M(V, Y, mu):
    """
    Parameters
    ----------
    V              : (q, m)-array
                     first m principal components
    Y              : (p,m)-array
                     Principal Component Coefficients 
    mu             : (q)-array
                     Average per column 
    Returns
    -------
    Mm              : (p, q)-array
                     "Modified" M
    """    
    V_t = V.T
    Z = np.dot(Y,V_t)
    
    # Sumamos de vuelta el vector promedio 
    Mm = Z + mu
    return Mm


## Preguntas

Para responder las preguntas a continuación, debe implementar las siguientes funciones. *Basta con que entregue la función. No responda la pregunta textualmente, recuerde que no se deben hacer prints ni pedir inputs en la tarea.*

**(10 puntos)** 1. Obtenga la matriz $V$ mediante los dos siguientes métodos: realice PCA usando la matriz de covarianza y realice PCA aplicando SVD, ¿cuál sería el tiempo de computación de cada uno? Deberá crear la función ```faster(t1, t2)``` que reciba los tiempos obtenidos con ```time_PCA_COV(M, m)``` y ```time_PCA_SVD(M, m)``` que indique si el primer tiempo es menor que el segundo o no. *Basta con que entregue la función. No responda la pregunta textualmente, recuerde que no se deben hacer prints ni pedir inputs en la tarea.*

In [26]:
# Esta función de no debe ser modificada.
def PCA_COV(M, m):
    """
    Parameters
    ----------
    M             : (p, q) array
                    M real matrix
    m              : int
                    Number of components
    Returns
    -------
    V             : (q, m)-array
                     first m principal components
    Y              : (p,m)-array
                     Principal Component Coefficients  
    mu             : (q)-array
                     Average per column 
    """
    mu = np.mean(M, axis = 0)
    Z = M - mu
    # Computar la matriz de covarianza de Z
    cov_mat = np.dot(Z.T,Z)

    # Computar los valores y vectores propios usando numpy
    eig_vals, eig_vecs = np.linalg.eig(cov_mat)
    eig_vals = np.real(eig_vals)
    eig_vecs = np.real(eig_vecs)
    
    # Ordenar de forma descendente los vectores propios según sus valores propios asociados
    order = np.argsort(eig_vals)
    order = np.flip(order)
    eig_vals = eig_vals[order]
    eig_vecs = eig_vecs[:,order]
    
    eig_vals = eig_vals[:m]
    V = eig_vecs[:,:m] # V matrix
    Y = M.dot(V)
    return V, Y, mu

# Esta función de no debe ser modificada.
def time_PCA_COV(n, m):
    """
    Parameters
    ----------
    n             : int
                    coefficient to determine the shape of the random matrix
    m             : int
                    Number of components
    Returns
    -------
    tiempo         : Float
                    execution time
    """
    np.random.seed(0) # Seed para la generación de matrices aleatorias
    M = np.random.random((n,2*n))
    t1 = time.time()
    V, Y, mu = PCA_COV(M,m)
    t2 = time.time()
    tiempo = t2-t1
    return tiempo
    
# Esta función de no debe ser modificada.
def time_PCA_SVD(n, m):
    """
    Parameters
    ----------
    n             : int
                    coefficient to determine the shape of the random matrix
    m             : int
                    Number of components
    Returns
    -------
    tiempo        : Float
                    execution time
    """
    np.random.seed(0) # Seed para la generación de matrices aleatorias
    M = np.random.random((n,2*n))
    t1 = time.time()
    V, Y, mu = PCA_SVD(M,m)
    t2 = time.time()
    tiempo = t2-t1
    
    return tiempo

In [27]:
def faster(t1, t2):
    """
    Parameters
    ----------
    t1             : Float
                    PCA_COV execution time
    t2             : Float
                    PCA_SVD execution time
    Returns
    -------
    is_faster       : bool
                    comparisson between t1 and t2
    """
  
    is_faster = t1 < t2
    return is_faster

**(20 puntos)** 2. ¿Cual sería la tasa de compresión utilizando $m1$ y $m2$ componentes en la reducción? Para responder esta pregunta realice la función ```calc_compression_ratio(data_length, M_shape, m1, m2)``` que obtenga el costo de almacenamiento de aplicar PCA a ```M_Re``` y ```M_Im``` (la matriz de $m$ vectores principales $V$, los coeficientes proyectados $Y$, y el vector $\mu$ para cada matriz) dividido en el costo de almacenamiento del vector de datos originales (pista de audio) $data$. 
La función debe retornar la tasa de compresión obtenida del siguiente cálculo:
$$ compression\_ratio = \left(1-\frac{memoria\_comprimida}{memoria\_original} \right)*100 $$
Notar que la memoria requerida es básicamente la cantidad de coeficientes que tiene que almacer en cada caso, considerando que en ambos casos se usará _double precision_ para almacenar cada coeficiente.

*Basta con que entregue la función. No responda la pregunta textualmente, recuerde que no se deben hacer prints ni pedir inputs en la tarea.* 

In [28]:
def calc_compression_ratio(data_length, M_shape, m1, m2):
    """
    Parameters
    ----------
    data_length          : length of 1-D array                                      
                          Length of the original NumPy array with WAV file data
    M_shape              : tuple
                          Complex M matrix Dimensions
    m1                   : int
                          Number of components to reduce M_Re real matrix
    m2                   : int
                          Number of components to reduce M_Im real matrix
    Returns
    -------
    compression_ratio  : Float
                         Compression ratio
    """
    #data_length = cant datos de mi audio original
    # M_shape = shape de la matriz original
    # m1 = cant componentes a reducir en la parte real
    # m2 = cant componentes a reducir en la parte imaginaria
    
    p, q = M_shape
    
    # Para M_Re (p,q)
    length_V_Re = q * m1
    length_Y_Re = p * m1
    length_mu_Re = q
    
    length_M_Re = length_V_Re + length_Y_Re + length_mu_Re
    
    # Para M_Im (p,q)
    length_V_Im = q * m2
    length_Y_Im = p * m2
    length_mu_Im = q
    
    length_M_Im = length_V_Im + length_Y_Im + length_mu_Im

    memoria_comprimida = length_M_Re + length_M_Im
    memoria_original = data_length
    
    compression_ratio = (1 - (memoria_comprimida/memoria_original))*100
    
    return compression_ratio

**(35 puntos)** 3. Utilizando el método más rápido encontrado en la pregunta 1 (PCA_COV o PCA_SVD) 
¿Cuál sería la combinación de valores $m1$ y $m2$ de componentes principales tal que se cumpla que el audio tenga un error menor o igual a ```quality_error``` y que al mismo tiempo se maximice el ```compression_ratio```? 
Es decir, debe encontrar la combinación de $m1$ y $m2$ que cumpla con el ```quality_error``` definido pero que requiera los mínimos valores posibles de $m1$ y $m2$.
Claramente la solución posiblemente no es única, pero usted debe entregar la que usted obtenga y que cumple con el ```quality_error``` y los menores valores de $m1$ y $m2$. Realice la función ```components_values(data, quality_error)``` que compute lo anteriormente mencionado.

Para esto construya las funciones ```compression_algorithm(data, m1, m2)``` que realice la compresión al audio original con las funciones creadas anteriormente, y ```decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2)``` que a partir de la compresión realizada pueda recuperar el formato de audio. 
El error de calidad entre el audio original y el audio reducido con $m1$ y $m2$ componentes se definiráá como la norma 2 de la diferencia entre los audios, es decir,
$$\mbox{error}=(\|\mbox{data}-\mbox{data}\_\mbox{modified}\|_2).$$
Asuma de que siempre existirá, por lo menos, una combinación que cumpla las restricciones. *Basta con que entregue la función. No responda la pregunta textualmente, recuerde que no se deben hacer prints ni pedir inputs en la tarea.*


In [29]:
def compression_algorithm(data, m1, m2):
    """
    Parameters
    ----------
    data               : 1-D array                                      
                       NumPy array with WAV file data
    m1                 : int
                       Number of components to reduce M_Re real matrix
    m2                 : int
                       Number of components to reduce M_Im real matrix

    Returns
    -------
    V1             : (q, m)-array
                     first m principal components from M_Re
    Y1              : (p,m)-array
                     Principal Component Coefficients from M_Re
    mu1            : (q)-array
                     Average per column from M_Re
    V2             : (q, m)-array
                     first m principal components from M_Im
    Y2              : (p,m)-array
                     Principal Component Coefficients from M_Im
    mu2             : (q)-array
                     Average per column from M_Im
                
    """
    #Matriz compleja
    M = STFT(data)
    
    #Separamos parte real y parte imaginaria
    M_Re, M_Im = Split(M)
    
    #Calculamos el PCA con SVD para cada matriz con sus respectivo m componente
    V1, Y1, mu1 = PCA_SVD(M_Re, m1)
    V2, Y2, mu2 = PCA_SVD(M_Im, m2)
   
    return V1, Y1, mu1, V2, Y2, mu2

def decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2):
    """
    Parameters
    ----------
    V1             : (q, m)-array
                     first m principal components from M_Re
    Y1              : (p,m)-array
                     Principal Component Coefficients from M_Re
    mu1            : (q)-array
                     Average per column from M_Re
    V2             : (q, m)-array
                     first m principal components from M_Im
    Y2              : (p,m)-array
                     Principal Component Coefficients from M_Im
    mu2             : (q)-array
                     Average per column from M_Im

    Returns
    -------
    data_modified   : 1-D array                                      
                      NumPy array with WAV file data
    """
    #PCA para matriz real e imaginaria
    Mm_R = PCA_M(V1, Y1, mu1) 
    Mm_Im = PCA_M(V2, Y2, mu2) 
    
    #Merge para las matrices modificadas
    Mm = Merge(Mm_R, Mm_Im)
    
    #Usamos ISTFT para volver a la data del .wav modificada
    data_modified = ISTFT(Mm)
    return data_modified


def components_values(data, quality_error):
    """
    Parameters
    ----------
    data               : 1-D array                                      
                       NumPy array with WAV file data
    quality_error :    : float
                         maximum quality error allowed

    Returns
    -------
    m1                 : int
                       Number of components to reduce M_Re real matrix
    m2                 : int
                       Number of components to reduce M_Im real matrix
    """ 
    i, j = 0, 0
        
    while(True):
        if (i > 129 or j > 129):
            return (129,129)
        else:
            V1, Y1, mu1, V2, Y2, mu2 = compression_algorithm(data, i, j)
            data_modified = decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2)
            a = data-data_modified
            error = np.linalg.norm(a)


            if (error > quality_error): # si el error no se sobrepasa se sigue iterando
                    i += 5
                    j += 5

            else: # si el error se sobrepasa, hay que acotar el rango
                i -= 1
                V1, Y1, mu1, V2, Y2, mu2 = compression_algorithm(data, i, j)
                data_modified = decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2)
                a = data-data_modified
                error = np.linalg.norm(a)
                
                while(error < quality_error):
                    if(i != 1):
                        i -= 1
                        V1, Y1, mu1, V2, Y2, mu2 = compression_algorithm(data, i, j)
                        data_modified = decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2)
                        a = data-data_modified
                        error = np.linalg.norm(a)
                    else:
                        break


                j -= 1
                V1, Y1, mu1, V2, Y2, mu2 = compression_algorithm(data, i, j)
                data_modified = decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2)
                a = data-data_modified
                error = np.linalg.norm(a)
                
                while(error < quality_error):
                    if(j != 1):
                        j -= 1
                        V1, Y1, mu1, V2, Y2, mu2 = compression_algorithm(data, i, j)
                        data_modified = decompression_algorithm(V1, Y1, mu1, V2, Y2, mu2)
                        a = data-data_modified
                        error = np.linalg.norm(a)
                    else:
                        break
                m1, m2 = i,j      
                valores_comp = []


                while(m1 <= m2):
                    ratio = calc_compression_ratio(len(data), (129, 18751), m1, m2)
                    valores_comp.append((ratio, (m1,m2)))
                    m1+=1
                    m2-=1

                ratio = calc_compression_ratio(len(data), (129, 18751), m1, m2)
                valores_comp.append((ratio, (m1,m2)))

                break
    
    index = valores_comp.index(max(valores_comp,key=lambda item:item[0]))
    
    m1, m2 = valores_comp[index][1]
    return m1, m2


# Referencias

Si corresponde
